Importing Datasets

In [ ]:
import pandas as pd

pendle_tweets_df = pd.read_json("./dataset/pendle.json")
pendle_llm_analysis_df = pd.read_csv("./dataset/pendle_llm_analysis.csv")
pendle_coin_df = pd.read_csv("./dataset/pendle_ohlcv_30m.csv")

tenx_coins_df = pd.read_csv("./dataset/10x_coins.csv")
not_tenx_coins_df = pd.read_csv("./dataset/not_10x_coins.csv")

In [ ]:
len(pendle_tweets_df),len(pendle_llm_analysis_df)

In [ ]:
pendle_tweets_df_non = pendle_tweets_df.dropna()

In [ ]:
pendle_tweets_df_non.shape

In [ ]:
pendle_tweets_df.head()

In [ ]:
pendle_llm_analysis_df.head()

In [ ]:
pendle_coin_df.head()

In [ ]:
# Explanation:
#  - Fetching All Coins Mentioned
#  - Filtering out all Tweets to include atleast pendle in the coins_mentions column 

import ahocorasick
import pandas as pd
import re
import jieba  # For Chinese text segmentation
import unicodedata

# Preprocess tenx_df and non_tenx_df: drop NaN values
tenx_df = tenx_coins_df.dropna(subset=["name", "id", "symbol", "screen_name"])
non_tenx_df = not_tenx_coins_df.dropna(subset=["name", "id", "symbol", "screen_name"])

# Function to build Aho-Corasick trie
def build_ac_trie(word_dict):
    """
    Builds an Aho-Corasick Trie (automaton) for fast multi-pattern string matching.

    The Aho-Corasick algorithm is an efficient method for searching multiple keywords 
    in a given text simultaneously. It constructs a Trie structure from a given dictionary 
    of keywords and then transforms it into an automaton that supports fast lookups.

    Steps:
    1. Insert each keyword from `word_dict` into the Trie.
    2. Convert the Trie into an Aho-Corasick automaton with failure links, allowing 
       efficient backtracking when mismatches occur.
    3. The resulting automaton enables linear-time matching of multiple words in a text.

    Parameters:
    word_dict (dict): A dictionary where keys are words (patterns to match) and values 
                      are associated identifiers.

    Returns:
    ahocorasick.Automaton: A compiled Aho-Corasick Trie ready for pattern matching.
    """
    trie = ahocorasick.Automaton()
    for key, value in word_dict.items():
        trie.add_word(key, value)
    trie.make_automaton()
    return trie

# Create mappings for tenx and non_tenx using 'name', 'symbol', and 'screen_name' columns
name_to_id_10x = {k.lower(): v.lower() for k, v in zip(tenx_df["name"], tenx_df["id"])}
symbol_to_id_10x = {k.lower(): v.lower() for k, v in zip(tenx_df["symbol"], tenx_df["id"])}
screen_name_to_id_10x = {"@" + k.lower(): v.lower() for k, v in zip(tenx_df["screen_name"], tenx_df["id"])}

name_to_id_non10x = {k.lower(): v.lower() for k, v in zip(non_tenx_df["name"], non_tenx_df["id"])}
symbol_to_id_non10x = {k.lower(): v.lower() for k, v in zip(non_tenx_df["symbol"], non_tenx_df["id"])}
screen_name_to_id_non10x = {"@" + k.lower(): v.lower() for k, v in zip(non_tenx_df["screen_name"], non_tenx_df["id"])}

# Merge both datasets' mappings
all_coin_mappings = {**name_to_id_10x, **symbol_to_id_10x, **screen_name_to_id_10x,
                      **name_to_id_non10x, **symbol_to_id_non10x, **screen_name_to_id_non10x}

# Common English words to filter out
common_words = {"about", "again", "all", "an", "and", "any", "are", "as", "at", "bad", "be", "big", "but", "by", "can", 
                "different", "do", "early", "every", "for", "from", "good", "has", "high", "how", "if", "in", "is", "it", 
                "just", "late", "like", "long", "low", "me", "more", "most", "much", "my", "new", "not", "now", "old", "on", 
                "one", "only", "or", "other", "out", "over", "own", "short", "so", "that", "the", "this", "to", "under", "up", 
                "way", "we", "well", "what", "when", "where", "why", "will", "with", "would", "you", "young", "your"}

# Filter out short and ambiguous names
filtered_mappings = {k: v for k, v in all_coin_mappings.items() if k not in common_words and len(k) > 2}

# Build Aho-Corasick trie
ac_trie = build_ac_trie(filtered_mappings)

# Define regex patterns
hashtag_pattern = re.compile(r'#([A-Za-z0-9]+)')
mention_pattern = re.compile(r'@([A-Za-z0-9_]+)')
dollar_pattern = re.compile(r'\$([A-Za-z0-9]+)')

# Function to detect non-English text
def contains_non_english(text):
    return any(unicodedata.category(char)[0] not in ('L', 'N') for char in text)

# Function to tokenize text
def tokenize_text(text):
    if contains_non_english(text):
        return jieba.lcut(text)  # Use jieba for Chinese/Japanese text
    else:
        return re.findall(r'\b[a-zA-Z0-9-]+\b', text)  # Normal word extraction

# Function to extract coin IDs from text
def extract_coin_ids(text):
    coin_ids = set()
    text_lower = text.lower()

    # Extract hashtags
    for match in hashtag_pattern.findall(text_lower):
        if match in filtered_mappings:
            coin_ids.add(filtered_mappings[match])

    # Extract mentions
    for match in mention_pattern.findall(text_lower):
        match_lower = "@" + match
        if match_lower in filtered_mappings:
            coin_ids.add(filtered_mappings[match_lower])

    # Extract tickers
    for match in dollar_pattern.findall(text_lower):
        if match in filtered_mappings:
            coin_ids.add(filtered_mappings[match])

    # Tokenize text based on language type
    words = tokenize_text(text_lower)

    # Use Aho-Corasick Trie to match words
    for word in words:
        if word in filtered_mappings:
            coin_ids.add(filtered_mappings[word])

    return list(coin_ids)

# Apply function to extract mentions
pendle_tweets_df["coin_mentions"] = pendle_tweets_df["fullText"].astype(str).apply(extract_coin_ids)

# Ensure proper filtering of truly empty coin_mentions
tweets_without_mentions = pendle_tweets_df[
    pendle_tweets_df["coin_mentions"].apply(lambda x: isinstance(x, list) and not x)
]

# Display sample results
tweets_without_mentions[["id", "fullText", "coin_mentions"]].head()

In [ ]:
len(pendle_tweets_df["coin_mentions"]) , len(pendle_tweets_df[pendle_tweets_df["coin_mentions"].apply(lambda x: "pendle" in x)])

In [ ]:
# Extracting tweets where atleast pendle is mentioned
pendle_mentioned_df = pendle_tweets_df[pendle_tweets_df["coin_mentions"].apply(lambda x: "pendle" in x)]

In [ ]:
pendle_mentioned_df.columns

Extracting the usernames from author column

In [ ]:
# Function to extract username from dictionary
def extract_username(user_info):
    try:
        return user_info.get("userName", None)  # Extract the username
    except (ValueError, SyntaxError):
        return None

# Apply extraction function to the username column
pendle_tweets_df["username"] = pendle_tweets_df["author"].apply(extract_username)

# Display the cleaned data
pendle_tweets_df[["id","fullText","username"]].head()

In [ ]:
# Function to extract username from dictionary
def extract_username(user_info):
    try:
        return user_info.get("userName", None)  # Extract the username
    except (ValueError, SyntaxError):
        return None

# Apply extraction function to the username column
pendle_mentioned_df["username"] = pendle_mentioned_df["author"].apply(extract_username)

# Display the cleaned data
pendle_mentioned_df[["id","fullText","username"]].head()

SubTask 1: For each account, find the time taken between the first mention of the coin, and the time taken by the coin to reach the 2x, 3x, and 5x.

In [ ]:
# Getting the first tweets of each user after storing out the createdAt
pendle_mentioned_df["createdAt"] = pd.to_datetime(pendle_mentioned_df["createdAt"]).dt.tz_localize(None)

# Group by username, sort by createdAt, and get the first tweet date per user
first_tweets = pendle_mentioned_df.sort_values("createdAt").groupby("username").first().reset_index()

# Extract only relevant columns
first_tweets = first_tweets[["username", "createdAt"]]

In [ ]:
first_tweets.head()

In [ ]:
first_tweets["rounded_timestamp"] = first_tweets["createdAt"].dt.floor("30T")

first_tweets

In [ ]:
pendle_coin_df["TIMESTAMP"] = pd.to_datetime(pendle_coin_df["TIMESTAMP"]).dt.tz_localize(None)

# Merge datasets again after fixing the timezone mismatch
merged_data = first_tweets.merge(pendle_coin_df, left_on="rounded_timestamp", right_on="TIMESTAMP", how="left")[["username", "createdAt", "rounded_timestamp","CLOSE"]]

merged_data["CLOSE"].fillna(0.4, inplace=True)

# Display the final output
merged_data.head()

In [ ]:
import numpy as np

# Ensure timestamps are in datetime format
merged_data["createdAt"] = pd.to_datetime(merged_data["createdAt"])
pendle_coin_df["TIMESTAMP"] = pd.to_datetime(pendle_coin_df["TIMESTAMP"])

# Sort price data for efficient searching
pendle_coin_df = pendle_coin_df.sort_values(by=["TIMESTAMP"])

# Convert to numpy arrays for fast vectorized operations
timestamps = pendle_coin_df["TIMESTAMP"].values
prices = pendle_coin_df["CLOSE"].values

# Vectorized function to find days to reach 2x, 3x, and 5x
def find_days_to_targets_fast(created_at, start_price):
    target_prices = np.array([start_price * 2, start_price * 3, start_price * 5])  # 2x, 3x, 5x
    idxs = np.searchsorted(prices, target_prices, side="left")  # Find first occurrence

    days = []
    for idx in idxs:
        if idx < len(timestamps):
            time_diff = (timestamps[idx] - created_at).days
            days.append(time_diff if time_diff >= 0 else None)  # Ensure no negatives
        else:
            days.append(0)  # No valid future price found

    return days

# Apply function efficiently
merged_data[["Days_to_2x", "Days_to_3x", "Days_to_5x"]] = np.vstack(
    merged_data.apply(lambda row: find_days_to_targets_fast(row["rounded_timestamp"], row["CLOSE"]), axis=1)
)

# Display result
merged_data.head()

In [ ]:
for col in ["Days_to_2x", "Days_to_3x", "Days_to_5x"]:
    merged_data[col + "_norm"] = 1 - (merged_data[col] - merged_data[col].min()) / (merged_data[col].max() - merged_data[col].min())

# Calculate the final weighted score
merged_data["time_to_5x_movement"] = (0.6 * merged_data["Days_to_5x_norm"] + 
                     0.25 * merged_data["Days_to_3x_norm"] + 
                     0.15 * merged_data["Days_to_2x_norm"])

In [ ]:
merged_data.head()

In [ ]:
pendle_coin_df['TIMESTAMP'] = pd.to_datetime(pendle_coin_df['TIMESTAMP'])
pendle_coin_df.set_index('TIMESTAMP', inplace=True)

In [ ]:
# Ensure datetime index
pendle_coin_df.index = pd.to_datetime(pendle_coin_df.index)

# Calculate percentage changes for CLOSE and VOLUME
pendle_coin_df['CLOSE_pct_change'] = pendle_coin_df['CLOSE'].pct_change() * 100
pendle_coin_df['VOLUME_pct_change'] = pendle_coin_df['VOLUME'].pct_change() * 100

# Compute relaxed surge thresholds (80th percentile)
close_threshold_80 = pendle_coin_df['CLOSE_pct_change'].quantile(0.80)
volume_threshold_80 = pendle_coin_df['VOLUME_pct_change'].quantile(0.80)

# Identify surges using relaxed thresholds
pendle_coin_df['surge'] = (
    (pendle_coin_df['CLOSE_pct_change'] >= close_threshold_80) &
    (pendle_coin_df['VOLUME_pct_change'] >= volume_threshold_80)
)

# Find contiguous surge periods
pendle_coin_df['surge_shift'] = pendle_coin_df['surge'].shift(1, fill_value=False)
pendle_coin_df['start'] = pendle_coin_df['surge'] & ~pendle_coin_df['surge_shift']  # Start of a new surge
pendle_coin_df['end'] = ~pendle_coin_df['surge'] & pendle_coin_df['surge_shift']    # End of a surge

# Extract start and end timestamps, ensuring minimum surge duration
surge_periods = []
start_time = None
min_surge_duration = pd.Timedelta(hours=1)  # Minimum surge duration
max_surge_duration = pd.Timedelta(hours=10)  # Maximum surge duration

for timestamp, row in pendle_coin_df.iterrows():
    if row['start']:
        start_time = timestamp
    if row['end'] and start_time is not None:
        duration = timestamp - start_time
        if duration >= min_surge_duration:
            surge_periods.append((start_time, timestamp))
        start_time = None

# Handle case where last surge doesn't have an end yet
if start_time is not None and pendle_coin_df['surge'].iloc[-1]:
    end_time = pendle_coin_df.index[-1]
    duration = end_time - start_time
    if duration >= min_surge_duration:
        surge_periods.append((start_time, end_time))

# Print surge periods
for start, end in surge_periods:
    print(f"Surge Period: {start.strftime('%Y-%m-%d %H:%M:%S')} to {end.strftime('%Y-%m-%d %H:%M:%S')}")

print("Total Surge Periods:", len(surge_periods))

SubTask2: Once you have found the surges, you need to calculate on average how much time before a surge, the account tweets about it. (Bullish sentiment).

In [ ]:
# Finding tweets that have a bullish sentiment
bullish_only_tweets = pendle_llm_analysis_df[pendle_llm_analysis_df["signal_classification"] == "bullish"]

bullish_tweets_ids = bullish_only_tweets["id"].to_list()

bullish_tweets_df = pendle_mentioned_df[pendle_mentioned_df["id"].isin(bullish_tweets_ids)]

In [ ]:
bullish_tweets_df.head()

In [ ]:
# Convert 'createdAt' to datetime and sort
bullish_tweets_df['createdAt'] = pd.to_datetime(bullish_tweets_df['createdAt'])
bullish_tweets_df = bullish_tweets_df.sort_values(by=['username', 'createdAt'])

# Convert surge periods to DataFrame with proper datetime format
surge_df = pd.DataFrame(surge_periods, columns=['surge_start', 'surge_end'])
surge_df['surge_start'] = pd.to_datetime(surge_df['surge_start'])
surge_df['surge_end'] = pd.to_datetime(surge_df['surge_end'])

# Find the last tweet before each surge for each user using a more efficient approach
results = []

# Create a dictionary for quick lookup of last tweets before surge
user_tweet_dict = {}
for username, group in bullish_tweets_df.groupby('username'):
    user_tweet_dict[username] = group['createdAt'].values

# Process each surge period
for _, surge_row in surge_df.iterrows():
    surge_start = surge_row['surge_start']
    
    # Process each user for this surge period
    for username, tweet_times in user_tweet_dict.items():
        # Find last tweet before surge using numpy which is much faster
        mask = tweet_times < surge_start
        if mask.any():
            last_tweet = tweet_times[mask].max()
            days_before = (surge_start - pd.Timestamp(last_tweet)).days
        else:
            days_before = 0
        
        results.append({'username': username, 'days_before_surge': days_before})

# Convert results to DataFrame and calculate average
df_days = pd.DataFrame(results)
avg_days_df = df_days.groupby('username', as_index=False)['days_before_surge'].mean()

# Display the results
avg_days_df

In [ ]:
avg_days_df["days_before_surge"].describe()

Defining the LookBack window to be 3 days and then calculating the metrics

In [ ]:
# Set lookback window (3 days)
LOOKBACK_HOURS = 72

# Ensure createdAt is datetime
bullish_tweets_df['createdAt'] = pd.to_datetime(bullish_tweets_df['createdAt'])

# Convert surge periods to DataFrame
surge_df = pd.DataFrame(surge_periods, columns=['surge_start', 'surge_end'])
surge_df['surge_start'] = pd.to_datetime(surge_df['surge_start'])
surge_df['surge_end'] = pd.to_datetime(surge_df['surge_end'])

# Initialize lists to store results
tweet_metrics = []

# Calculate metrics for each surge
for idx, surge in surge_df.iterrows():
    lookback_start = surge['surge_start'] - pd.Timedelta(hours=LOOKBACK_HOURS)
    
    window_tweets = bullish_tweets_df[
        ((bullish_tweets_df['createdAt'] >= lookback_start) & 
         (bullish_tweets_df['createdAt'] <= surge['surge_end']))
    ]
    
    if window_tweets.empty:
        continue
        
    for username, user_surge_tweets in window_tweets.groupby('username'):
        user_surge_tweets['lead_time'] = (surge['surge_start'] - user_surge_tweets['createdAt']).dt.total_seconds() / (24*3600)
        
        pre_surge_tweets = user_surge_tweets[user_surge_tweets['createdAt'] < surge['surge_start']]
        avg_lead_time = pre_surge_tweets['lead_time'].mean() if not pre_surge_tweets.empty else 0
        
        surge_mention = len(user_surge_tweets)
        
        interaction_cols = ['retweetCount', 'replyCount', 'likeCount', 'quoteCount', 'bookmarkCount']
        user_surge_tweets['engagement_score'] = user_surge_tweets[interaction_cols].sum(axis=1) / user_surge_tweets['viewCount'].clip(lower=1)
        avg_engagement = user_surge_tweets['engagement_score'].mean()
        
        tweet_metrics.append({
            'username': username,
            'lead_time': avg_lead_time,
            'surge_mention': surge_mention,
            'engagement_score': avg_engagement,
            'surge_count': 1
        })

if not tweet_metrics:
    print("No tweets found within lookback windows")
    result_df = pd.DataFrame(columns=['username', 'normalized_score'])
else:
    # Convert to DataFrame and aggregate by username
    metrics_df = pd.DataFrame(tweet_metrics)
    
    # Aggregate metrics by username
    user_metrics = metrics_df.groupby('username').agg({
        'lead_time': 'mean',
        'surge_mention': 'sum',
        'engagement_score': 'mean',
        'surge_count': 'sum'
    }).reset_index()
    
    # Calculate conviction score
    total_surges = len(surge_df)
    user_metrics['conviction_score'] = user_metrics['surge_count'] / total_surges
    
    # Manual min-max normalization for each metric
    norm_cols = ['lead_time', 'surge_mention', 'conviction_score', 'engagement_score']
    
    for col in norm_cols:
        # Check if all values are the same
        if user_metrics[col].min() == user_metrics[col].max():
            user_metrics[f'normalized_{col}'] = 1.0
        else:
            # Min-max normalization: (x - min) / (max - min)
            min_val = user_metrics[col].min()
            max_val = user_metrics[col].max()
            user_metrics[f'normalized_{col}'] = (user_metrics[col] - min_val) / (max_val - min_val)
    
    # # Calculate final combined score (equal weighting)
    # normalized_cols = [f'normalized_{col}' for col in norm_cols]
    # user_metrics['normalized_score'] = user_metrics[normalized_cols].mean(axis=1)
    
    # Final result with username and normalized score
    result_df = user_metrics[['username', 'normalized_lead_time', 'normalized_surge_mention', 'normalized_conviction_score', 'normalized_engagement_score']]

In [ ]:
result_df["lead_time_before_price_surges"] = (
    0.5 * result_df["normalized_lead_time"] +
    0.2 * result_df["normalized_surge_mention"] +
    0.15 * result_df["normalized_conviction_score"] +
    0.15 * result_df["normalized_engagement_score"]
)

In [ ]:
result_df.head()

early_detection Scoring

Creating list of all unique users

In [ ]:
len(pendle_tweets_df["username"].unique())

In [ ]:
len(result_df["username"]), len(merged_data["username"])

In [ ]:
import pandas as pd

# Step 1: Extract unique usernames from pendle_tweets_df
unique_usernames = pendle_tweets_df["username"].unique()

# Step 2: Initialize dictionary with default values
user_dict = {user: {"time_to_5x_movement": 0, "lead_time_before_price_surges": 0} for user in unique_usernames}

# Step 3: Update the dictionary with values from merged_data
for _, row in merged_data.iterrows():
    username = row["username"]
    if username in user_dict:
        user_dict[username]["time_to_5x_movement"] = row["time_to_5x_movement"]

# Step 4: Update the dictionary with values from results_df
for _, row in result_df.iterrows():
    username = row["username"]
    if username in user_dict:
        user_dict[username]["lead_time_before_price_surges"] = row["lead_time_before_price_surges"]

# Step 5: Convert the dictionary to a DataFrame
final_df = pd.DataFrame.from_dict(user_dict, orient="index").reset_index()
final_df.rename(columns={"index": "username"}, inplace=True)

# Display the first few rows
final_df.head()

In [ ]:
len(final_df["username"])

In [ ]:
# Calculate early detection score
final_df["early_detection"] = (
    final_df["time_to_5x_movement"] * 0.5 +
    final_df["lead_time_before_price_surges"] * 0.5
)

# Sort in descending order based on early detection
final_df = final_df.sort_values(by="early_detection", ascending=False)

final_df.head()

Tweet Consistency Task

In [ ]:
pendle_mentioned_df.columns

In [ ]:
pendle_mentioned_df["rounded_timestamp"] = pendle_mentioned_df["createdAt"].dt.floor("30T")

Tweets Frequency

In [ ]:
# Create an empty DataFrame to store tweet frequencies
tweet_frequencies = []

# Define lookback period for pre-surge tweets
lookback_period = pd.Timedelta(hours=72)

# Process each surge period
for start, end in surge_periods:
    # Filter tweets during surge
    surge_tweets = pendle_mentioned_df[(pendle_mentioned_df['createdAt'] >= start) & (pendle_mentioned_df['createdAt'] <= end)]
    surge_counts = surge_tweets.groupby("username").size().reset_index(name="insurge_tweets_frequency")
    
    # Filter tweets 72 hours before surge
    pre_surge_tweets = pendle_mentioned_df[(pendle_mentioned_df['createdAt'] >= start - lookback_period) & (pendle_mentioned_df['createdAt'] < start)]
    pre_surge_counts = pre_surge_tweets.groupby("username").size().reset_index(name="presurge_tweets_frequency")
    
    # Merge results to ensure all usernames are considered
    tweet_frequency = pd.merge(pre_surge_counts, surge_counts, on="username", how="outer").fillna(0)
    
    # Append results
    tweet_frequencies.append(tweet_frequency)

# Combine results into a single DataFrame
tweet_frequency_result = pd.concat(tweet_frequencies, ignore_index=True)

# **Aggregate by username to get total tweets across all surge periods**
tweet_frequency_result = tweet_frequency_result.groupby("username", as_index=False).sum()

# Sort by in-surge tweet frequency
tweet_frequency_result.sort_values(by="insurge_tweets_frequency", ascending=False, inplace=True)

tweet_frequency_result.head()

Buy Tweets Ratio

In [ ]:
# Define lookback period for pre-surge tweets
lookback_period = pd.Timedelta(hours=72)

# Getting IDs of all Buy tweets
buy_tweets_ids = pendle_llm_analysis_df[pendle_llm_analysis_df["call_to_action"] == "buy"]["id"].to_list()

# Create an empty list to store buy tweet ratios
buy_ratios = []

# Function to compute buy tweet ratio
def compute_buy_tweet_ratio(df):
    total_tweets = df.groupby("username")["id"].count()
    buy_tweets = df[df["id"].isin(buy_tweets_ids)].groupby("username")["id"].count()
    return (buy_tweets / total_tweets).fillna(0)

# Temporary storage for merging all results
all_pre_surge_ratios = []
all_in_surge_ratios = []

# Calculate buy tweet ratio before and during surges
for start, end in surge_periods:
    # Filter tweets during surge
    surge_tweets = pendle_mentioned_df[(pendle_mentioned_df['createdAt'] >= start) & (pendle_mentioned_df['createdAt'] <= end)]
    
    # Filter tweets 72 hours before surge
    pre_surge_tweets = pendle_mentioned_df[(pendle_mentioned_df['createdAt'] >= start - lookback_period) & (pendle_mentioned_df['createdAt'] < start)]
    
    # Compute buy tweet ratios
    pre_surge_buy_ratio = compute_buy_tweet_ratio(pre_surge_tweets)
    in_surge_buy_ratio = compute_buy_tweet_ratio(surge_tweets)
    
    # Store results for later aggregation
    all_pre_surge_ratios.append(pre_surge_buy_ratio)
    all_in_surge_ratios.append(in_surge_buy_ratio)

# Combine all periods and sum them across users
pre_surge_buy_ratios = pd.concat(all_pre_surge_ratios, axis=1).sum(axis=1)
in_surge_buy_ratios = pd.concat(all_in_surge_ratios, axis=1).sum(axis=1)

# Merge into final DataFrame
buy_ratio_result = pd.DataFrame({
    "username": pre_surge_buy_ratios.index.union(in_surge_buy_ratios.index),
    "presurge_buy_tweets_ratio": pre_surge_buy_ratios,
    "insurge_buy_tweets_ratio": in_surge_buy_ratios
}).fillna(0)

# Sort by In-Surge Buy Tweets Ratio (highest first)
buy_ratio_result = buy_ratio_result.sort_values(by="insurge_buy_tweets_ratio", ascending=False)
buy_ratio_result.reset_index(drop=True, inplace=True)

buy_ratio_result.head()

In [ ]:
buy_ratio_result["presurge_buy_tweets_ratio"].describe()

Bullish Tweets Ratio

In [ ]:
# Define lookback period for pre-surge tweets
lookback_period = pd.Timedelta(hours=72)

# Getting IDs of all Bullish tweets
bullish_tweets_ids = pendle_llm_analysis_df[pendle_llm_analysis_df["signal_classification"] == "bullish"]["id"].to_list()

# Function to compute bullish tweet ratio
def compute_bullish_tweet_ratio(df):
    total_tweets = df.groupby("username")["id"].count()
    bullish_tweets = df[df["id"].isin(bullish_tweets_ids)].groupby("username")["id"].count()
    return (bullish_tweets / total_tweets).fillna(0)

# Temporary storage for merging all results
all_pre_surge_ratios = []
all_in_surge_ratios = []

# Calculate bullish tweet ratio before and during surges
for start, end in surge_periods:
    # Filter tweets during surge
    surge_tweets = pendle_mentioned_df[(pendle_mentioned_df['createdAt'] >= start) & (pendle_mentioned_df['createdAt'] <= end)]
    
    # Filter tweets 72 hours before surge
    pre_surge_tweets = pendle_mentioned_df[(pendle_mentioned_df['createdAt'] >= start - lookback_period) & (pendle_mentioned_df['createdAt'] < start)]
    
    # Compute bullish tweet ratios
    pre_surge_bullish_ratio = compute_bullish_tweet_ratio(pre_surge_tweets)
    in_surge_bullish_ratio = compute_bullish_tweet_ratio(surge_tweets)
    
    # Store results for later aggregation
    all_pre_surge_ratios.append(pre_surge_bullish_ratio)
    all_in_surge_ratios.append(in_surge_bullish_ratio)

# Combine all periods and sum them across users
pre_surge_bullish_ratios = pd.concat(all_pre_surge_ratios, axis=1).sum(axis=1)
in_surge_bullish_ratios = pd.concat(all_in_surge_ratios, axis=1).sum(axis=1)

# Merge into final DataFrame
bullish_ratio_result = pd.DataFrame({
    "username": pre_surge_bullish_ratios.index.union(in_surge_bullish_ratios.index),
    "presurge_bullish_tweets_ratio": pre_surge_bullish_ratios,
    "insurge_bullish_tweets_ratio": in_surge_bullish_ratios
}).fillna(0)

# Sort by In-Surge Bullish Tweets Ratio (highest first)
bullish_ratio_result = bullish_ratio_result.sort_values(by="insurge_bullish_tweets_ratio", ascending=False)
bullish_ratio_result.reset_index(drop=True, inplace=True)

bullish_ratio_result.head()

Hype Scoring

In [ ]:
tweets_analysis_merged_df = pendle_mentioned_df.merge(pendle_llm_analysis_df, on="id", how="left")  # 'left' keeps all records in df

# Now df contains the analysis for each tweet
tweets_analysis_merged_df.head()

In [ ]:
len(tweets_analysis_merged_df)

In [ ]:
tweets_analysis_merged_df['in_surge_period'] = tweets_analysis_merged_df['createdAt'].apply(lambda x: any(start <= x <= end for start, end in surge_periods))
surge_tweets = tweets_analysis_merged_df[tweets_analysis_merged_df['in_surge_period']]
non_surge_tweets = tweets_analysis_merged_df[~tweets_analysis_merged_df['in_surge_period']]

In [ ]:
# Define the scoring criteria
hype_scores = {"low": 0, "normal": 1, "high": 2}

# Map hype classification to numerical scores
surge_tweets["hype_score"] = surge_tweets["hype_classification"].map(hype_scores)

# Group by username and sum the hype scores
final_hype_scores = surge_tweets.groupby("username")["hype_score"].sum().reset_index()

# Sort the results from highest to lowest
final_hype_scores = final_hype_scores.sort_values(by="hype_score", ascending=False)

# Display the top results
final_hype_scores.head()

In [ ]:
# Define the scoring criteria
hype_scores = {"low": 0, "normal": 1, "high": 2}

# Map hype classification to numerical scores
non_surge_tweets["hype_score"] = non_surge_tweets["hype_classification"].map(hype_scores)

# Group by username and sum the hype scores
final_hype_non_surge = non_surge_tweets.groupby("username")["hype_score"].sum().reset_index()

# Sort the results from highest to lowest
final_hype_non_surge = final_hype_non_surge.sort_values(by="hype_score", ascending=False)

# Display the top results
final_hype_non_surge.head()

Manipulative Tweets Ratio in Surge

In [ ]:
# Count total tweets and manipulative tweets per username
manipulative_ratio = surge_tweets.groupby("username").agg(
    total_tweets=("id", "count"),
    manipulative_tweets=("crypto_manipulative_words", "sum")
)

# Compute the ratio
manipulative_ratio["manipulative_tweets_ratio"] = (
    manipulative_ratio["manipulative_tweets"] / manipulative_ratio["total_tweets"]
)

# Sort in descending order
manipulative_ratio = manipulative_ratio.sort_values(by="manipulative_tweets_ratio", ascending=False)
manipulative_ratio = manipulative_ratio[["manipulative_tweets_ratio"]].reset_index()

# Display results
manipulative_ratio.head()

In [ ]:
# Count total tweets and manipulative tweets per username
manipulative_ratio_non_surge = non_surge_tweets.groupby("username").agg(
    total_tweets=("id", "count"),
    manipulative_tweets=("crypto_manipulative_words", "sum")
)

# Compute the ratio
manipulative_ratio_non_surge["manipulative_tweets_ratio"] = (
    manipulative_ratio_non_surge["manipulative_tweets"] / manipulative_ratio_non_surge["total_tweets"]
)

# Sort in descending order
manipulative_ratio_non_surge = manipulative_ratio_non_surge.sort_values(by="manipulative_tweets_ratio", ascending=False)
manipulative_ratio_non_surge = manipulative_ratio_non_surge[["manipulative_tweets_ratio"]].reset_index()

# Display results
manipulative_ratio_non_surge.head()

In [ ]:
all_unique_users = pendle_tweets_df["username"].unique()

len(all_unique_users)

In [ ]:
# Create a reference dataframe from unique users
unique_users_df = pd.DataFrame({"username": all_unique_users})  # Ensure this is a complete list

# List of dataframes to merge
dfs = [tweet_frequency_result, buy_ratio_result, bullish_ratio_result, final_hype_scores, manipulative_ratio]

# # Start merging from the unique users dataframe
# merged_df = reduce(lambda left, right: pd.merge(left, right, on="username", how="left"), [unique_users_df] + dfs)

merged_df = unique_users_df.copy()
for df in dfs:
    merged_df = pd.merge(merged_df, df, on="username", how="left")


# Fill missing values with 0
merged_df = merged_df.fillna(0)

# Display the final result
merged_df.head()

In [ ]:
len(unique_users_df)

In [ ]:
len(all_unique_users), len(merged_df["username"])

Normalizing Columns

In [ ]:
# Select only numeric columns for normalization
numeric_columns = merged_df.select_dtypes(include=["number"]).columns

# Apply Min-Max Scaling manually
for col in numeric_columns:
    min_val = merged_df[col].min()
    max_val = merged_df[col].max()
    if max_val != min_val:  # Avoid division by zero
        merged_df[col] = (merged_df[col] - min_val) / (max_val - min_val)
    else:
        merged_df[col] = 0  # If all values are the same, set column to 0


merged_df.head()

In [ ]:
merged_df.columns

In [ ]:
merged_df["consistency_score"] = (
    merged_df["presurge_buy_tweets_ratio"] * 0.25 +
    merged_df["insurge_tweets_frequency"] * 0.1 +
    merged_df["presurge_bullish_tweets_ratio"] * 0.1 +
    merged_df["insurge_bullish_tweets_ratio"] * 0.05 +
    merged_df["presurge_tweets_frequency"] * 0.1 +
    merged_df["insurge_buy_tweets_ratio"] * 0.1 +
    merged_df["hype_score"] * 0.15 +
    merged_df["manipulative_tweets_ratio"] * 0.15
)

merged_df.sort_values(by="consistency_score",ascending=False,inplace=True)

merged_df[["username","consistency_score"]].head()

In [ ]:
merged_df.sort_values(by="username",ascending=True, inplace=True)
final_df.sort_values(by="username",ascending=True, inplace=True)

timing_and_relevance = pd.DataFrame()

timing_and_relevance["username"] = merged_df["username"]
timing_and_relevance["score"] = merged_df["consistency_score"] * 0.4 + final_df["early_detection"] * 0.6

timing_and_relevance.sort_values(by="score", ascending=False, inplace=True)

timing_and_relevance.head()

In [ ]:
len(timing_and_relevance["username"])